# Transport Canberra API

In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import requests

## API Base URL

In [2]:
# API endpoint
api_token = 'C57DB8'
api_url_base = 'http://siri.nxtbus.act.gov.au:11000/'

In [3]:
# environment variables
city_bus_plt_1 = 3401
city_bus_plt_2 = 3402
max_stop_visits = 4
max_text_length = 160

## API Endpoints

In [4]:
endpoints = ['sm', 'vm', 'pt', 'et']

service_name = {
    'poll status': 'status.xml',
    'manage data subscription': 'subscription.xml',
    'poll data': 'polldata.xml',
    'report data ready': 'dataready.xml',
    'direct delivery': 'directdelivery.xml',
    'request for data': 'service.xml'
}

## API URL Generator

In [5]:
api_url = '{}{}/{}/{}'.format(api_url_base, api_token, endpoints[0], service_name['request for data'])

print(api_url)
response = requests.get(api_url)

http://siri.nxtbus.act.gov.au:11000/C57DB8/sm/service.xml


In [6]:
if response.status_code == 200:
    print('Connection Successful')
else:
    print('Check connection parameters')

Connection Successful


## XML Header

In [7]:
header = "<?xml version='1.0' encoding='ISO-8859-1' standalone='yes'?>"

## XML Body

In [8]:
import xmltodict
import dicttoxml
import pprint
import json
from datetime import datetime

In [9]:
timestamp = str(datetime.now()).split(' ')
timestamp[1] = timestamp[1].split('.')[0]
timestamp[1] = timestamp[1]+'+10:00'
timestamp = 'T'.join(timestamp)
print(timestamp)

2019-05-18T10:20:30+10:00


In [10]:
my_xml = """
<Siri version="1.3" xmlns:ns2="http://www.ifopt.org.uk/acsb" xmlns="http://www.siri.org.uk/siri"
xmlns:ns4="http://datex2.eu/schema/2_0RC1/2_0" xmlns:ns3="http://www.ifopt.org.uk/ifopt">
    <ServiceRequest>
        <RequestTimestamp></RequestTimestamp>
        <RequestorRef></RequestorRef>
        <StopMonitoringRequest version="2.0">
            <RequestTimestamp></RequestTimestamp>
            <MonitoringRef></MonitoringRef>
            <MaximumStopVisits></MaximumStopVisits>
            <MaximumTextLength></MaximumTextLength>
        </StopMonitoringRequest>
    </ServiceRequest>
</Siri>
"""
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(json.dumps(xmltodict.parse(my_xml)))

('{"Siri": {"@version": "1.3", "@xmlns:ns2": "http://www.ifopt.org.uk/acsb", '
 '"@xmlns": "http://www.siri.org.uk/siri", "@xmlns:ns4": '
 '"http://datex2.eu/schema/2_0RC1/2_0", "@xmlns:ns3": '
 '"http://www.ifopt.org.uk/ifopt", "ServiceRequest": {"RequestTimestamp": '
 'null, "RequestorRef": null, "StopMonitoringRequest": {"@version": "2.0", '
 '"RequestTimestamp": null, "MonitoringRef": null, "MaximumStopVisits": null, '
 '"MaximumTextLength": null}}}}')


In [11]:
my_xml = """
    <audience>
      <id what="attribute">123</id>
      <name>Shubham</name>
    </audience>
"""
my_dict = xmltodict.parse(my_xml)
print(my_dict['audience']['id'])
print(my_dict['audience']['id']['@what'])

OrderedDict([('@what', 'attribute'), ('#text', '123')])
attribute


In [12]:
sm_xml = {'Siri': {'@version': '2.0', 
          '@xmlns': 'http://www.siri.org.uk/siri',
          '@xmlns:ns2': 'http://www.ifopt.org.uk/acsb',
          '@xmlns:ns3': 'http://www.ifopt.org.uk/ifopt',
          '@xmlns:ns4': 'http://datex2.eu/schema/2_0RC1/2_0',
          'ServiceRequest': { 'RequestTimestamp': timestamp,
                              'RequestorRef': api_token,
                              'StopMonitoringRequest': { '@version': '2.0',
                                                         'RequestTimestamp': timestamp,
                                                         'MonitoringRef': city_bus_plt_1,
                                                         'MaximumStopVisits': max_stop_visits,
                                                         'MaximumTextLength': max_text_length
                                                       }
                            }
}}

In [13]:
sm_xml.keys()

dict_keys(['Siri'])

In [14]:
sm_xml['Siri'].keys()

dict_keys(['@version', '@xmlns', '@xmlns:ns2', '@xmlns:ns3', '@xmlns:ns4', 'ServiceRequest'])

In [15]:
sm_xml['Siri']['ServiceRequest'].keys()

dict_keys(['RequestTimestamp', 'RequestorRef', 'StopMonitoringRequest'])

In [16]:
sm_xml['Siri']['ServiceRequest']['StopMonitoringRequest'].keys()

dict_keys(['@version', 'RequestTimestamp', 'MonitoringRef', 'MaximumStopVisits', 'MaximumTextLength'])

In [17]:
print(xmltodict.unparse(sm_xml, pretty=True))

<?xml version="1.0" encoding="utf-8"?>
<Siri version="2.0" xmlns="http://www.siri.org.uk/siri" xmlns:ns2="http://www.ifopt.org.uk/acsb" xmlns:ns3="http://www.ifopt.org.uk/ifopt" xmlns:ns4="http://datex2.eu/schema/2_0RC1/2_0">
	<ServiceRequest>
		<RequestTimestamp>2019-05-18T10:20:30+10:00</RequestTimestamp>
		<RequestorRef>C57DB8</RequestorRef>
		<StopMonitoringRequest version="2.0">
			<RequestTimestamp>2019-05-18T10:20:30+10:00</RequestTimestamp>
			<MonitoringRef>3401</MonitoringRef>
			<MaximumStopVisits>4</MaximumStopVisits>
			<MaximumTextLength>160</MaximumTextLength>
		</StopMonitoringRequest>
	</ServiceRequest>
</Siri>


In [18]:
sm_xml

{'Siri': {'@version': '2.0',
  '@xmlns': 'http://www.siri.org.uk/siri',
  '@xmlns:ns2': 'http://www.ifopt.org.uk/acsb',
  '@xmlns:ns3': 'http://www.ifopt.org.uk/ifopt',
  '@xmlns:ns4': 'http://datex2.eu/schema/2_0RC1/2_0',
  'ServiceRequest': {'RequestTimestamp': '2019-05-18T10:20:30+10:00',
   'RequestorRef': 'C57DB8',
   'StopMonitoringRequest': {'@version': '2.0',
    'RequestTimestamp': '2019-05-18T10:20:30+10:00',
    'MonitoringRef': 3401,
    'MaximumStopVisits': 4,
    'MaximumTextLength': 160}}}}

In [19]:
headers = {'Content-Type': 'application/xml'}

In [20]:
r = requests.post(api_url, data=xmltodict.unparse(sm_xml), headers=headers)

In [21]:
r = r.content.decode('utf-8')

In [22]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(json.dumps(xmltodict.parse(r)))

('{"Siri": {"@xmlns": "http://www.siri.org.uk/siri", "@xmlns:ns2": '
 '"http://www.ifopt.org.uk/acsb", "@xmlns:ns3": '
 '"http://www.ifopt.org.uk/ifopt", "@xmlns:ns4": '
 '"http://datex2.eu/schema/2_0RC1/2_0", "@xmlns:xsi": '
 '"http://www.w3.org/2001/XMLSchema-instance", "@version": "2.0", '
 '"@xsi:schemaLocation": "http://www.siri.org.uk/siri ../siri.xsd", '
 '"ServiceDelivery": {"ResponseTimestamp": "2019-05-18T10:20:31+10:00", '
 '"ProducerRef": "ACT", "Status": "true", "StopMonitoringDelivery": '
 '{"@version": "2.0", "ResponseTimestamp": "2019-05-18T10:20:31+10:00", '
 '"MonitoredStopVisit": [{"RecordedAtTime": "2019-05-18T10:20:31.148+10:00", '
 '"ValidUntilTime": "2019-05-18T10:30:25+10:00", "MonitoringRef": "3401", '
 '"ClearDownRef": "450", "MonitoredVehicleJourney": {"LineRef": "ACT_57", '
 '"DirectionRef": "B", "FramedVehicleJourneyRef": {"DataFrameRef": '
 '"2019-05-18", "DatedVehicleJourneyRef": "689-00006-1"}, "PublishedLineName": '
 '"57", "ExternalLineRef": "57", "Ope

In [23]:
response_dict = json.loads(json.dumps(xmltodict.parse(r)))

### Response

In [24]:
response_dict.keys()

dict_keys(['Siri'])

In [25]:
response_dict['Siri'].keys()

dict_keys(['@xmlns', '@xmlns:ns2', '@xmlns:ns3', '@xmlns:ns4', '@xmlns:xsi', '@version', '@xsi:schemaLocation', 'ServiceDelivery'])

In [26]:
response_dict['Siri']['ServiceDelivery'].keys()

dict_keys(['ResponseTimestamp', 'ProducerRef', 'Status', 'StopMonitoringDelivery'])

### Response: Stop Monitoring Delivery

In [27]:
response_dict['Siri']['ServiceDelivery']['StopMonitoringDelivery'].keys()

dict_keys(['@version', 'ResponseTimestamp', 'MonitoredStopVisit'])

In [30]:
response_dict['Siri']['ServiceDelivery']['StopMonitoringDelivery']['MonitoredStopVisit'][0].keys()

dict_keys(['RecordedAtTime', 'ValidUntilTime', 'MonitoringRef', 'ClearDownRef', 'MonitoredVehicleJourney'])

In [31]:
monitored_stop_visit = response_dict['Siri']['ServiceDelivery']['StopMonitoringDelivery']['MonitoredStopVisit']

[{'RecordedAtTime': '2019-05-18T10:20:31.148+10:00',
  'ValidUntilTime': '2019-05-18T10:30:25+10:00',
  'MonitoringRef': '3401',
  'ClearDownRef': '450',
  'MonitoredVehicleJourney': {'LineRef': 'ACT_57',
   'DirectionRef': 'B',
   'FramedVehicleJourneyRef': {'DataFrameRef': '2019-05-18',
    'DatedVehicleJourneyRef': '689-00006-1'},
   'PublishedLineName': '57',
   'ExternalLineRef': '57',
   'OperatorRef': 'ACT',
   'ProductCategoryRef': 'standard',
   'VehicleFeatureRef': ['Bike Rack', 'Wheel Chair'],
   'OriginShortName': 'city',
   'DestinationDisplayAtOrigin': 'Woden',
   'DestinationName': 'Woden',
   'DestinationShortName': 'wodi',
   'OriginAimedDepartureTime': '2019-05-18T10:28:00+10:00',
   'DestinationAimedArrivalTime': '2019-05-18T11:10:00+10:00',
   'Monitored': 'true',
   'InCongestion': 'false',
   'BlockRef': '15046',
   'CourseOfJourneyRef': '0',
   'VehicleRef': '450',
   'MonitoredCall': {'StopPointRef': '3401',
    'VisitNumber': '1',
    'VehicleAtStop': 'false',
